# OpenAI agents SDK

These are the basic packages needed to use openai SDK

See: https://openai.github.io/openai-agents-python/quickstart/

## Install openai agents

```shell
uv add openai-agents
```

In [ ]:
# Import the necessary packages

from dotenv import load_dotenv
from agents import Agent, Runner, trace

In [ ]:
# Load environment variables from a .env file if present
load_dotenv(override=True)

## Creating Agents

In [ ]:
# OpenAI Agent creation

instructions = """
You are a joke teller. Tell me a joke.
"""
model = "gpt-4o-mini"

agent = Agent(name="Jokester", instructions=instructions, model=model)


In [ ]:
# Run the agent

prompt = "Tell a joke about Fruits and Vegetables"

with trace("Telling a joke"):
    result = await Runner.run(agent, prompt)
    print(result.final_output)

## Structured outputs via Pydantic

Agents can return structured data using Pydantic models instead of plain text. This ensures type safety and consistent output formatting.

```shell
uv add pydantic
```

In [ ]:
# Define structured output

from pydantic import BaseModel

class JokeResponse(BaseModel):
    joke: str

In [ ]:
# Use structured output as the output type for the agent
another_agent = Agent(
    name="Jokester-2",
    instructions=instructions,
    model=model,
    output_type=JokeResponse
)

# Run the agent
prompt = "Tell a joke about software developers"

with trace("Telling a joke with structured output"):
    result = await Runner.run(another_agent, prompt)
    print(result.final_output)

## Function Tools

Agents or LLMs can use tools to be more useful. It connects LLM to data and other functionalities.

In [ ]:
# Use the decorator @function_tool to create a function tool that can be used by an agent

from agents import function_tool

@function_tool
def write_to_file(filename: str = "output.txt", content: str = "") -> bool:
    with open(filename, "w") as f:
        f.write(content)
    return True

file_writer_agent = Agent(
    name="FileWriter",
    instructions="You write content to a file. Use the tool to write to a file.",
    model="gpt-4o-mini",
    tools=[write_to_file]
)

# Run the agent
prompt = "Write 'Hello, World!' to a file named hello.txt"
with trace("Writing to a file using an agent"):
    result = await Runner.run(file_writer_agent, prompt)
    print(result.final_output)

## Agents interacting with each other

Agents can collaborate through multiple interaction patterns:

- Manual interaction: Explicitly pass outputs between agents
- Tool-based interaction: One agent uses another as a function tool
- Handoffs: Structured transfer of control between agents

Each approach offers different levels of control and automation for multi-agent workflows.

### Manual agent interaction

In [ ]:
# Direct agent interaction

from pydantic import BaseModel

class CritiqueResponse(BaseModel):
    joke: str
    score: int
    explanation: str


critic_agent: Agent[CritiqueResponse] = Agent(
    name="Critic",
    instructions="You are a joke critique. You judge jokes based on how funny they are on a scale of 1 to 10. You must always provide a score and a short explanation. You also explain how why a joke is funny or not funny. The input to you is a joke.",
    model="gpt-4o-mini",
    output_type=CritiqueResponse
)

joker_agent = Agent(
    name="Jokester",
    instructions="You tell jokes. It may or may not be funny and it's up to you to decide. Make sure that jokes are not offensive.",
    model="gpt-4o-mini"
)

prompt = "Tell a random joke about planets"

with trace("Joke and Critique Handoff"):
    joke = await Runner.run(joker_agent, prompt)
    print("Joke:", joke.final_output)
    result = await Runner.run(critic_agent, joke.final_output)
    critique: CritiqueResponse = result.final_output
    print("Critique:", critique.model_dump_json(indent=2))




### Agent as a tool

In [ ]:
from agents import function_tool

@function_tool
def write_to_file(filename: str = "output.txt", content: str = "") -> bool:
    with open(filename, "w") as f:
        f.write(content)
    return True

file_writer_agent = Agent(
    name="FileWriter",
    instructions="You write content to a file. Use the tool to write to a file.",
    model="gpt-4o-mini",
    tools=[write_to_file]
)

file_writer_agent_as_tool = file_writer_agent.as_tool(
    tool_name="FileWriterTool",
    tool_description="Use this tool to write content to a file. It has two parameters: filename and content. Filename is the name of the file to write to. Content is the content to write to the file. Both parameters are required."
)
joker_agent = Agent(
    name="Jokester",
    instructions="You tell jokes. It may or may not be funny and it's up to you to decide. Make sure that jokes are not offensive.",
    tools=[file_writer_agent_as_tool],
    model="gpt-4o-mini"
)

prompt = "Tell a random joke about planets and write it to a file called joke.txt"
with trace("Joke and File Writing"):
    result = await Runner.run(joker_agent, prompt)
    print(result.final_output)



## Agent handoffs

Agent handoffs enable structured transfer of control between agents during a conversation. Unlike manual interaction where you explicitly coordinate agents, handoffs allow an agent to automatically transfer the conversation to another agent when specific conditions are met. This creates seamless multi-agent workflows where agents can specialize in different tasks and collaborate autonomously.

In [ ]:
from agents import Agent, Runner, trace
from pydantic import BaseModel

class JokeEvaluation(BaseModel):
    joke: str
    is_appropriate: bool
    needs_improvement: bool
    feedback: str
    

# Create a joke evaluator agent
joke_evaluator = Agent(
    name="JokeEvaluator", 
    instructions="You evaluate jokes for appropriateness and quality. If a joke is inappropriate or needs improvement, provide feedback.",
    model="gpt-4o-mini",
    output_type=JokeEvaluation,
    handoff_description="Evaluate a joke.",
)

# Create a joke generator agent
joke_generator = Agent(
    name="JokeGenerator",
    instructions="You generate jokes based on user requests. Keep them clean and appropriate.",
    model="gpt-4o-mini",
    handoffs=[joke_evaluator]
)

prompt = "Generate a joke about programming. Always evaluate it for appropriateness and quality."

with trace("Joke Creation with Handoffs"):
    result = await Runner.run(joke_generator, prompt)
    print(result.final_output)